In [8]:
# install packages
!pip install langchain langgraph ollama requests beautifulsoup4 playwright --break-system-packages --quiet

# restart kernel to use new packages

In [1]:
pip install ollama --break-system-packages --quiet

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "lsv2_xxxxxxxx"

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from ollama import chat
from ollama import ChatResponse

# use ollama model 

response: ChatResponse = chat(model='llama3.2', messages=[
  {
    'role': 'user',
    'content': 'What is the Stock Market? Give insighs about indian stock Market',
  },
])
print(response['message']['content'])
# or access fields directly from the response object
print(response.message.content)

The stock market, also known as the equity market, is a platform where companies raise capital by issuing shares of stock to the public, and investors can buy and sell these shares in hopes of making a profit. The stock market provides an opportunity for individuals, businesses, and institutions to invest in companies and potentially benefit from their growth.

Here's an overview of how the stock market works:

**Key Players:**

1. **Issuers**: Companies that want to raise capital by issuing shares of stock.
2. **Investors**: Individuals, institutional investors, and other entities that buy and sell shares of stock.
3. **Exchanges**: Platforms where buyers and sellers meet to trade shares, such as the Bombay Stock Exchange (BSE) in India.

**How it Works:**

1. **Initial Public Offering (IPO)**: Companies issue shares of stock to the public for the first time, typically to raise capital for expansion or growth.
2. **Share Prices**: The value of each share is determined by supply and de

In [4]:
# use open source model 

from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model='deepseek-r1:1.5b', messages=[
  {
    'role': 'user',
    'content': 'What is the Stock Market? Give insighs about indian stock Market',
  },
])
print(response['message']['content'])
# or access fields directly from the response object
print(response.message.content)

<think>
Okay, so I'm trying to understand what a stock market is and how it works. From what I remember, the stock market is where people buy and sell stocks. But I want to know more than that. Let me try to break this down.

First, what exactly is a stock? I think it's something like a share of ownership in a company. So when someone buys a stock, they're committing themselves to owning part of the company. Now, where does this happen? I believe it's on stock exchanges, which are typically places where these stocks are traded. Examples include the New York Stock Exchange (NYSE) and the Indian Stock Exchange (NSE). 

I'm not entirely sure about the difference between a scrip and a share capital. Maybe scrips are smaller, like 1-5 rupee shares, while share capital is for larger shares of a certain amount. That makes sense because sometimes companies need more shares to operate or expand.

Now, how do these stocks work? Do they pay dividends, which I've heard about before? Dividends are 

In [5]:
from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model='deepseek-r1:1.5b', messages=[
  {
    'role': 'user',
    'content': """A manufacturing plant produces cars.  

- Each **Car** requires **6 hours of labor** and **10 units of raw material** to produce.  

The factory has a total of **180 hours of labor** and **200 units of raw material** available per week.  

1. Formulate a system of equations to represent this situation.  
2. Solve for the maximum number of Cars that can be produced within the given constraints.  
4. If a Car sells for **$25,000**, calculate the **maximum revenue possible** under these constraints."""  
  },
])

print(response['message']['content'])
# or access fields directly from the response object
print(response.message.content)



<think>
Alright, so I have this problem about a manufacturing plant producing cars. Let me try to break it down step by step.

First, let's understand what is being asked. We need to figure out how many cars can be produced given certain constraints on labor hours and raw material. There are also two parts: one for solving the system of equations and then another part where we calculate maximum revenue. Hmm, okay.

Starting with part 1: Formulating a system of equations. 

Each car requires 6 hours of labor and 10 units of raw material. So if I let x be the number of cars produced, then:

- The total labor required would be 6x hours.
- The total raw material needed would be 10x units.

The factory has a maximum of 180 labor hours and 200 raw material available each week. Therefore, these constraints can be translated into inequalities:

6x ≤ 180 (for labor)
10x ≤ 200 (for raw material)

But actually, since we are looking for the maximum number of cars that can be produced, it's probabl

In [6]:
%reload_ext autoreload
%autoreload 2

In [7]:
import ollama
import json
import requests
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage
from typing import TypedDict, Annotated, Sequence
import operator


In [8]:
llama_model = "llama3.2:latest"
deepseek_model = "deepseek-r1:1.5b"

In [9]:
# Use ollama.chat() instead of model.invoke() and explicitly mention in English or it will get chinese response
def analyze_financials(state):
    """Use DeepSeek to analyze financial data in English."""
    messages = f"Analyze the financial data in **English**:\n\nITC: {state['itc_data']}\nHUL: {state['hul_data']}"
    response = ollama.chat(model=deepseek_model, messages=[{"role": "user", "content": messages}])
    return {"insights": response["message"]["content"]}


def compare_results(state):
    messages = f"Compare ITC India and Hindustan Unilever Financial results based on these insights in English:\n{state['insights']}"
    response = ollama.chat(model=llama_model, messages=[{"role": "user", "content": messages}])
    return {"comparison": response["message"]["content"]}

In [10]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[str], operator.add]
    itc_data: dict
    hul_data: dict
    insights: str
    comparison: str


def scrape_financials(state):
    """Fetch financial data for ITC India & HUL."""
    urls = {
        "itc": "https://www.moneycontrol.com/financials/itc/balance-sheetvi/itc",
        "hul": "https://www.moneycontrol.com/financials/hindustanunilever/balance-sheetvi/hu",
    }

    def fetch_data(url):
        response = requests.get(url)
        if response.status_code == 200:
            return {"data": response.text}  # Ideally, parse HTML to extract key values
        return {"error": "Failed to fetch"}

    return {
        "itc_data": fetch_data(urls["itc"]),
        "hul_data": fetch_data(urls["hul"]),
    }

In [11]:
# --- Build the Graph ---
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("scraper", scrape_financials)
workflow.add_node("analyzer", analyze_financials)
workflow.add_node("comparator", compare_results)

# Set flow
workflow.set_entry_point("scraper")
workflow.add_edge("scraper", "analyzer")
workflow.add_edge("analyzer", "comparator")
workflow.add_edge("comparator", END)

# Compile the workflow
app = workflow.compile()

# --- Run the system ---
inputs = {"messages": ["Fetch and compare ITC and HUL's last two quarters of financial results and respond strictly in English."]}
output = app.invoke(inputs)

print("Comparison Results:\n", output["comparison"])

Comparison Results:
 Based on the provided HTML code and analysis, I will compare ITC India and Hindustan Unilever's financial results. Please note that this is a general comparison and may not reflect the current market situation.

**Company Overview**

* ITC India: A leading diversified conglomerate with businesses in FMCG, agriculture, and renewable energy.
* Hindustan Unilever (HUL): A multinational consumer goods company with a presence in over 150 countries.

**Financial Results Comparison**

| Category | ITC India | Hindustan Unilever |
| --- | --- | --- |
| Revenue Growth | 10.5% YoY (2022-23) | 12.1% YoY (2022-23) |
| Profit Before Tax (PBT) | ₹2,434 Cr | ₹2,634 Cr |
| Net Profit | ₹1,444 Cr | ₹1,534 Cr |
| Earnings Per Share (EPS) | ₹17.35 | ₹18.55 |

**Key Performance Indicators**

* ITC India:
	+ FMCG segment revenue growth: 8.5% YoY
	+ Agriculture segment revenue growth: 15.6% YoY
	+ Renewable energy segment revenue growth: 20.0% YoY
* Hindustan Unilever:
	+ FMCG segment r